In [1]:
import pandas as pd
import numpy as np

import scanpy as sc
import biolord

import seaborn as sns
import matplotlib.pyplot as plt
import warnings
from scipy.stats import ttest_rel
import anndata

/home/icb/dominik.klein/mambaforge/envs/ot_pert_biolord/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
ood_split = 2

In [3]:
adata_train_path = f"/lustre/groups/ml01/workspace/ot_perturbation/data/sciplex/adata_train_{ood_split}.h5ad"
adata_test_path = f"/lustre/groups/ml01/workspace/ot_perturbation/data/sciplex/adata_test_{ood_split}.h5ad"
adata_ood_path = f"/lustre/groups/ml01/workspace/ot_perturbation/data/sciplex/adata_ood_{ood_split}.h5ad"

In [4]:
adata_train = sc.read(adata_train_path)
adata_test = sc.read(adata_test_path)
adata_ood = sc.read(adata_ood_path)

/home/icb/dominik.klein/mambaforge/envs/ot_pert_biolord/lib/python3.12/site-packages/anndata/__init__.py:51: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(
/home/icb/dominik.klein/mambaforge/envs/ot_pert_biolord/lib/python3.12/site-packages/anndata/__init__.py:51: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(
/home/icb/dominik.klein/mambaforge/envs/ot_pert_biolord/lib/python3.12/site-packages/anndata/__init__.py:51: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(


In [5]:
adata = anndata.concat((adata_train, adata_test, adata_ood), label="split", keys=["train", "test", "ood"])

/home/icb/dominik.klein/mambaforge/envs/ot_pert_biolord/lib/python3.12/site-packages/anndata/_core/anndata.py:1906: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [6]:
frac_valid = 0.1

def create_split(x):
    if x["split"] != "train":
        return "other"
    is_train = np.random.choice(2, p=[frac_valid, 1 - frac_valid])
    if is_train:
        return "train_train"
    return "train_valid"


adata.obs["new_split"] = adata.obs.apply(create_split, axis=1)

In [7]:
dose = adata.obs["dose"].astype("float") / np.max(adata.obs["dose"].astype("float")) # following biolord repr.

In [8]:
adata.obsm["ecfp_dose"] = np.concatenate((adata.obsm["ecfp"], dose.values[:,None]), axis=1)

In [9]:
biolord.Biolord.setup_anndata(
    adata,
    ordered_attributes_keys=["ecfp_dose"],
    categorical_attributes_keys=["cell_type"],
    retrieval_attribute_key=None,
)

INFO     Generating sequential column names                                                                        


In [10]:
module_params= dict(
    decoder_width = 4096,
    decoder_depth = 4,
    
    attribute_dropout_rate = 0.1,
    attribute_nn_width = 2048,
    attribute_nn_depth = 2,
    
    unknown_attribute_noise_param = 2e+1,
    unknown_attribute_penalty = 1e-1,
    gene_likelihood = "normal",
    n_latent_attribute_ordered = 256,
    n_latent_attribute_categorical = 3,
    reconstruction_penalty = 1e+4,
    use_batch_norm = False,
    use_layer_norm = False,)

trainer_params=dict(
    latent_lr = 1e-4,
    latent_wd = 1e-4,
    decoder_lr = 1e-4,
    decoder_wd = 1e-4,
    attribute_nn_lr = 1e-2,
    attribute_nn_wd = 4e-8,
    cosine_scheduler = True,
    scheduler_final_lr = 1e-5,
    step_size_lr = 45,
)

In [11]:
model = biolord.Biolord(
    adata=adata,
    n_latent=256,
    model_name="sciplex3",
    module_params=module_params,
    train_classifiers=False,
    split_key="new_split",
    train_split="train_train",
    valid_split="train_valid",
    test_split="other",
)

[rank: 0] Seed set to 0


In [12]:
model.train(
    max_epochs=200,
    batch_size=512,
    plan_kwargs=trainer_params,
    early_stopping=True,
    early_stopping_patience=20,
    check_val_every_n_epoch=10,
    num_workers=10,
    enable_checkpointing=False
)

/home/icb/dominik.klein/mambaforge/envs/ot_pert_biolord/lib/python3.12/site-packages/lightning/fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/icb/dominik.klein/mambaforge/envs/ot_pert_biol ...
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/icb/dominik.klein/mambaforge/envs/ot_pert_biolord/lib/python3.12/site-packages/lightning/fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/icb/dominik.klein/mambaforge/envs/ot_pert_biol ...
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/icb/dominik.klein/mambaforge/envs/ot_pert_biolord/lib/

Epoch 2/200:   0%|          | 1/200 [01:44<5:47:32, 104.79s/it, v_num=1, val_generative_mean_accuracy=0.331, val_generative_var_accuracy=-22.9, val_biolord_metric=-11.3, val_LOSS_KEYS.RECONSTRUCTION=277, val_LOSS_KEYS.UNKNOWN_ATTRIBUTE_PENALTY=233]

/home/icb/dominik.klein/mambaforge/envs/ot_pert_biolord/lib/python3.12/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([1, 2001])) that is different to the input size (torch.Size([2001])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/home/icb/dominik.klein/mambaforge/envs/ot_pert_biolord/lib/python3.12/site-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 10 worker processes in total. Our suggested max number of worker in current system is 6, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/home/icb/dominik.klein/mambaforge/envs/ot_pert_biolord/lib/python3.12/multiprocessin

Epoch 3/200:   1%|          | 2/200 [03:23<5:34:20, 101.32s/it, v_num=1, val_generative_mean_accuracy=0.328, val_generative_var_accuracy=-19.1, val_biolord_metric=-9.39, val_LOSS_KEYS.RECONSTRUCTION=278, val_LOSS_KEYS.UNKNOWN_ATTRIBUTE_PENALTY=211, generative_mean_accuracy=0, generative_var_accuracy=0, biolord_metric=0, reconstruction_loss=240, unknown_attribute_penalty_loss=1.03e+5]

/home/icb/dominik.klein/mambaforge/envs/ot_pert_biolord/lib/python3.12/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([1, 2001])) that is different to the input size (torch.Size([2001])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/home/icb/dominik.klein/mambaforge/envs/ot_pert_biolord/lib/python3.12/site-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 10 worker processes in total. Our suggested max number of worker in current system is 6, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/home/icb/dominik.klein/mambaforge/envs/ot_pert_biolord/lib/python3.12/multiprocessin

...

/home/icb/dominik.klein/mambaforge/envs/ot_pert_biolord/lib/python3.12/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([1, 2001])) that is different to the input size (torch.Size([2001])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/home/icb/dominik.klein/mambaforge/envs/ot_pert_biolord/lib/python3.12/site-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 10 worker processes in total. Our suggested max number of worker in current system is 6, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/home/icb/dominik.klein/mambaforge/envs/ot_pert_biolord/lib/python3.12/multiprocessin

Epoch 94/200:  47%|████▋     | 94/200 [2:26:48<2:45:32, 93.71s/it, v_num=1, val_generative_mean_accuracy=0.977, val_generative_var_accuracy=0.964, val_biolord_metric=0.97, val_LOSS_KEYS.RECONSTRUCTION=182, val_LOSS_KEYS.UNKNOWN_ATTRIBUTE_PENALTY=1.96e-10, generative_mean_accuracy=0, generative_var_accuracy=0, biolord_metric=0, reconstruction_loss=179, unknown_attribute_penalty_loss=1.02e+5]

/home/icb/dominik.klein/mambaforge/envs/ot_pert_biolord/lib/python3.12/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([1, 2001])) that is different to the input size (torch.Size([2001])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


/home/icb/dominik.klein/mambaforge/envs/ot_pert_biolord/lib/python3.12/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Monitored metric val_biolord_metric did not improve in the last 20 records. Best score: 0.973. Signaling Trainer to stop.


In [13]:
def bool2idx(x):
    """
    Returns the indices of the True-valued entries in a boolean array `x`
    """
    return np.where(x)[0]

def repeat_n(x, n):
    """
    Returns an n-times repeated version of the Tensor x,
    repetition dimension is axis 0
    """
    # copy tensor to device BEFORE replicating it n times
    device = "cuda" if torch.cuda.is_available() else "cpu"
    return x.to(device).view(1, -1).repeat(n, 1)


In [14]:
idx_test_control = np.where(
    (adata.obs["split"] == "test") & (adata.obs["control"] == 1)
)[0]

adata_test_control = adata[idx_test_control].copy()

idx_ood = np.where(((adata.obs["split"] == "ood") & (adata.obs["control"] == 0)))[0]

adata_ood = adata[idx_ood].copy()
dataset_ood = model.get_dataset(adata_ood)

INFO     AnnData object appears to be a copy. Attempting to transfer setup.                                        


In [15]:
dataset_ood = model.get_dataset(adata_ood)

In [16]:
import pandas as pd
from tqdm import tqdm

def compute_prediction(
    model,
    adata,
    dataset,
    adata_control,
    n_obs=500
):
    pert_categories_index = pd.Index(adata.obs["condition"].values, dtype="category")

    cl_dict = {
        torch.Tensor([0.]): "A549",
        torch.Tensor([1.]): "K562",
        torch.Tensor([2.]): "MCF7",
    }

    cell_lines = ["A549", "K562", "MCF7"]

    layer = "X" if "X" in dataset else "layers"
    predictions_dict = {}
    for cell_drug_dose_comb in tqdm(np.unique(pert_categories_index.values)
    ):
        cur_cell_line = cell_drug_dose_comb.split("_")[0]
        dataset_control = model.get_dataset(adata_test_control[adata_test_control.obs.cell_type == cur_cell_line])

        bool_category = pert_categories_index.get_loc(cell_drug_dose_comb)
        idx_all = bool2idx(bool_category)
        idx = idx_all[0]
                    
        dataset_comb = {}

        dataset_comb[layer] = dataset_control[layer].to(model.device)
        dataset_comb["ind_x"] = dataset_control["ind_x"].to(model.device)
        for key in dataset_control:
            if key not in [layer, "ind_x"]:
                dataset_comb[key] = repeat_n(dataset[key][idx, :], n_obs)

        stop = False
        for tensor, cl in cl_dict.items():
            if (tensor == dataset["cell_type"][idx]).all():
                if cl not in cell_lines:
                    stop = True
        if stop:
            continue
            
        pred_mean, pred_std = model.module.get_expression(dataset_comb)
        samples = torch.normal(pred_mean, pred_std)

        predictions_dict[cell_drug_dose_comb] = samples.detach().cpu().numpy()
    return predictions_dict

In [17]:
import torch
biolord_prediction = compute_prediction(
    model=model,
    adata=adata_ood,
    dataset=dataset_ood,
    adata_control=adata_test_control)

  0%|          | 0/540 [00:00<?, ?it/s]

INFO     Received view of anndata, making copy.                                                                    
INFO     Input AnnData not setup with scvi-tools. attempting to transfer AnnData setup                             


  0%|          | 1/540 [00:01<14:14,  1.58s/it]

INFO     Received view of anndata, making copy.                                                                    
INFO     Input AnnData not setup with scvi-tools. attempting to transfer AnnData setup                             
INFO     Received view of anndata, making copy.                                                                    
INFO     Input AnnData not setup with scvi-tools. attempting to transfer AnnData setup                             


  1%|          | 3/540 [00:01<04:08,  2.16it/s]

...

 99%|█████████▉| 536/540 [00:48<00:00, 14.44it/s]

INFO     Received view of anndata, making copy.                                                                    
INFO     Input AnnData not setup with scvi-tools. attempting to transfer AnnData setup                             
INFO     Received view of anndata, making copy.                                                                    
INFO     Input AnnData not setup with scvi-tools. attempting to transfer AnnData setup                             


100%|█████████▉| 538/540 [00:48<00:00, 14.27it/s]

INFO     Received view of anndata, making copy.                                                                    
INFO     Input AnnData not setup with scvi-tools. attempting to transfer AnnData setup                             
INFO     Received view of anndata, making copy.                                                                    
INFO     Input AnnData not setup with scvi-tools. attempting to transfer AnnData setup                             


100%|██████████| 540/540 [00:48<00:00, 11.06it/s]


In [18]:
import anndata as ad
all_data = []
conditions = []

for condition, array in biolord_prediction.items():
    all_data.append(array)
    conditions.extend([condition] * array.shape[0])

# Stack all data vertically to create a single array
all_data_array = np.vstack(all_data)

# Create a DataFrame for the .obs attribute
obs_data = pd.DataFrame({
    'condition': conditions
})

# Create the Anndata object
adata_ood_result = ad.AnnData(X=all_data_array, obs=obs_data)

/home/icb/dominik.klein/mambaforge/envs/ot_pert_biolord/lib/python3.12/site-packages/anndata/_core/anndata.py:183: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


In [19]:
adata_ood_result.write_h5ad(f"/lustre/groups/ml01/workspace/ot_perturbation/data/sciplex/biolord_output_ood_{ood_split}.h5ad")

In [20]:
1

1

In [21]:
idx_test_control = np.where(
    (adata.obs["split"] == "test") & (adata.obs["control"] == 1)
)[0]

adata_test_control = adata[idx_test_control].copy()

idx_test = np.where(((adata.obs["split"] == "test") & (adata.obs["control"] == 0)))[0]

adata_test = adata[idx_test].copy()
dataset_test = model.get_dataset(adata_test)

INFO     AnnData object appears to be a copy. Attempting to transfer setup.                                        


In [22]:
dataset_test = model.get_dataset(adata_test)

In [23]:
biolord_prediction = compute_prediction(
    model=model,
    adata=adata_test,
    dataset=dataset_test,
    adata_control=adata_test_control)

  0%|          | 0/1585 [00:00<?, ?it/s]

INFO     Received view of anndata, making copy.                                                                    
INFO     Input AnnData not setup with scvi-tools. attempting to transfer AnnData setup                             
INFO     Received view of anndata, making copy.                                                                    
INFO     Input AnnData not setup with scvi-tools. attempting to transfer AnnData setup                             


  0%|          | 2/1585 [00:00<02:01, 13.00it/s]

INFO     Received view of anndata, making copy.                                                                    
INFO     Input AnnData not setup with scvi-tools. attempting to transfer AnnData setup                             
INFO     Received view of anndata, making copy.                                                                    
INFO     Input AnnData not setup with scvi-tools. attempting to transfer AnnData setup                             


  0%|          | 4/1585 [00:00<01:56, 13.53it/s]

...

100%|█████████▉| 1582/1585 [02:11<00:00, 13.78it/s]

INFO     Received view of anndata, making copy.                                                                    
INFO     Input AnnData not setup with scvi-tools. attempting to transfer AnnData setup                             
INFO     Received view of anndata, making copy.                                                                    
INFO     Input AnnData not setup with scvi-tools. attempting to transfer AnnData setup                             


100%|█████████▉| 1584/1585 [02:11<00:00, 14.08it/s]

INFO     Received view of anndata, making copy.                                                                    
INFO     Input AnnData not setup with scvi-tools. attempting to transfer AnnData setup                             


100%|██████████| 1585/1585 [02:11<00:00, 12.06it/s]


In [24]:
all_data = []
conditions = []

for condition, array in biolord_prediction.items():
    all_data.append(array)
    conditions.extend([condition] * array.shape[0])

# Stack all data vertically to create a single array
all_data_array = np.vstack(all_data)

# Create a DataFrame for the .obs attribute
obs_data = pd.DataFrame({
    'condition': conditions
})

# Create the Anndata object
adata_test_result = ad.AnnData(X=all_data_array, obs=obs_data)

/home/icb/dominik.klein/mambaforge/envs/ot_pert_biolord/lib/python3.12/site-packages/anndata/_core/anndata.py:183: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


In [25]:
adata_test_result.write_h5ad(f"/lustre/groups/ml01/workspace/ot_perturbation/data/sciplex/biolord_output_test_{ood_split}.h5ad")